# 1. 에일리언 vs 프레데터 데이터셋
* https://www.kaggle.com/datasets/pmigdal/alien-vs-predator-images
* 케글 로그인 -> 우측 상단의 계정을 클릭 -> Your Profile -> 중앙에 Account를 클릭 -> API 항목에 Create New API Token -> kaggle.json이 다운로드 됨
* {"username":"yetzzang","key":"35d18a20f4029db5ddbf24bf1458c0da"}

In [60]:
import os

In [61]:
os.environ['KAGGLE_USERNAME'] = 'yetzzang'
os.environ['KAGGLE_KEY'] = "35d18a20f4029db5ddbf24bf1458c0da"

In [62]:
!kaggle datasets download -d pmigdal/alien-vs-predator-images

In [ ]:
!unzip -q alien-vs-predator-images.zip

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

# 2. 이미 증강 기법 (Image Augmentation)

* 원본 이미지(데이터)를 조작하여 원본과는 크고 작은 변화를 가진 이미지를 생성
  * 일반적으로 모델 성능이 좋아짐
  * 오버피팅을 방지
* https://pytorch.org/vision/master/transforms.html

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'validation':transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),

    ])
}

In [ ]:
def target_transforms(target):
  return torch.FloatTensor([target])


In [ ]:
image_datasets = {
    'train': datasets.ImageFolder('data/train', data_transforms['train'], target_transform=target_transforms),
    'validation':datasets.ImageFolder('data/validation', data_transforms['validation'], target_transform=target_transforms)
}

In [ ]:
dataloaders = {
    'train': DataLoader(
        image_datasets['train'],
        batch_size=32,
        shuffle=True
    ),
    'validation': DataLoader(
        image_datasets['validation'],
        batch_size=32,
        shuffle=False
    )
}

In [ ]:
print(len(image_datasets['train']), len(image_datasets['validation']))

In [ ]:
imgs, labels = next(iter(dataloaders['train']))

fig, axes = plt.subplots(4, 8, figsize=(16, 8))

for ax, img, label in zip(axes.flatten(), imgs, labels):
    ax.imshow(img.permute(1, 2, 0))
    ax.set_title(label.item())
    ax.axis('off')

# 3. 전이 학습 (Transfer Learning)
* 하나의 작업을 위해 훈련된 모델을 유사 작업 수행 모델의 시작점으로 활용하는 딥러닝 접근법
* 신경망은 처음부터 새로 학습하는 것보다 전이 학습을 통해 업데이트하고 재학습하는 편이 더 빠르고 간편함
* 전이 학습은 여러 응용 분야 중에서도 특히 검출, 영상 인식, 음성 인식, 검색 분야에 많이 사용

![](https://miro.medium.com/max/1400/1*9GTEzcO8KxxrfutmtsPs3Q.png)

### 3-1. 전이 학습의 고려할 점
* 크기: 모델 크기는 배포할 위치와 방법에 따라 달라짐
* 예측 속도와 정확도: 하드웨어 및 배치 크기와 같은 요소를 고려
![](https://kr.mathworks.com/discovery/transfer-learning/_jcr_content/mainParsys/image.adapt.full.medium.jpg/1634621302937.jpg)

# 4. 사전 학습된 ResNet50 모델
* [파이토치에서 제공하는 사전학습 모델들](https://pytorch.org/vision/stable/models.html)

![](https://i.stack.imgur.com/gI4zT.png)

### 4-1. 이미지넷(ImageNet)
* 이미지 데이터베이스
* 1000개의 동물과 사물 이미지를 포함

![](https://i.imgur.com/5Rz5s8e.png)

In [ ]:
model = models.resnet50(weights='IMAGENET1K_V1').to(device)
print(model)

### 4-2. Freeze Layers
* 특징을 뽑아내는 CNN의 앞쪽 컨볼루션 레이어들은 학습을 하지 않도록 설정
  * 이미 학습 된 애들은 그만 둘래
* 출력 부분의 레이어(fc)를 다시 설정하여 분류에 맞게 변경

In [ ]:
for param in model.parameters():
  param.requires_grad = False # 가져온 파라미터(W, b)를 업데이트 하지 않음

model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(),
    nn.Linear(128, 1),
    nn.Sigmoid()
).to(device)

print(model)

In [59]:
# 학습
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        sum_losses = 0
        sum_accs = 0

        for x_batch, y_batch in dataloaders[phase]:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(x_batch)

            loss = nn.BCELoss()(y_pred, y_batch)

            if phase == 'train':
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            sum_losses = sum_losses + loss.item()


            y_bool = (y_pred >= 0.5).float()
            acc = (y_batch == y_bool).float().sum() / len(y_batch) * 100
            sum_accs = sum_accs + acc.item()

        avg_loss = sum_losses / len(dataloaders[phase])
        avg_acc = sum_accs / len(dataloaders[phase])

        print(f'{phase:10s}: Epoch {epoch+1:4d}/{epochs} Loss: {avg_loss:.4f} Accuracy: {avg_acc: .2f}%')


train     : Epoch    1/10 Loss: 0.6170 Accuracy:  70.89%
validation: Epoch    1/10 Loss: 0.4827 Accuracy:  89.29%
train     : Epoch    2/10 Loss: 0.3906 Accuracy:  87.89%
validation: Epoch    2/10 Loss: 0.3173 Accuracy:  92.86%
train     : Epoch    3/10 Loss: 0.2898 Accuracy:  90.07%
validation: Epoch    3/10 Loss: 0.2543 Accuracy:  92.86%
train     : Epoch    4/10 Loss: 0.2192 Accuracy:  92.19%
validation: Epoch    4/10 Loss: 0.2336 Accuracy:  90.62%


KeyboardInterrupt: ignored